In [299]:
# Basic library
import nltk
import logging

import pandas as pd
import matplotlib.pyplot as plt

from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.tokenize.util import align_tokens
from nltk.tokenize.api import TokenizerI
from nltk.tokenize import word_tokenize
from recordlinkage.index import Block
import recordlinkage as rl
import recordlinkage
import jellyfish
import warnings
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


pd.set_option('display.max_columns', 100)

In [300]:
def2010 = pd.DataFrame()
def2011 = pd.DataFrame()
def2012 = pd.DataFrame()
def2013 = pd.DataFrame()
def2014 = pd.DataFrame()
def2015 = pd.DataFrame()
def2016 = pd.DataFrame()
def2017 = pd.DataFrame()
def2018 = pd.DataFrame()
def2019 = pd.DataFrame()
def2020 = pd.DataFrame()
def2021 = pd.DataFrame()
def2022 = pd.DataFrame()

In [302]:
#date = ['curp']
datasetsDef = ['seed2010_conv.csv', 'seed2011_conv.txt', 'seed2012_conv.txt', 'seed2013_conv.txt', 'seed2014_conv.txt', 'seed2015_conv.txt', 
                'seed2016_conv.txt', 'seed2017_conv.txt', 'seed2018_conv.txt', 'seed2019_conv.txt', 'seed2020_conv.txt', 'seed2021_conv.txt', 
                'seed2022_conv.txt']
anio = '2015'

bd = pd.read_csv('dgis_datos/defuncion/seed' + anio + '_conv.txt', delimiter='|', parse_dates=False, encoding='utf-8')
cols = ['edad_c', 'edad', 'nom_c', 'nac_dia', 'nac_mes', 'nac_anio', 'sexo', 'afiliacion', 'curp', 'edo_civil', 'escolar', 'ocupacion']
certific = ['cert_por', 'cert_ced', 'cert_nom', 'num_acta', 'informante']
residencia = ['res_ent', 'res_mun', 'res_loc', 'def_ent', 'def_mun', 'def_sitio', 'def_dia', 'def_mes', 'def_anio']

bd = bd[cols + residencia + certific]
#list(bd.loc[[10, 12]]['FECINISI'])

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

: 

In [ ]:
##########################Campos vacios##########################
print('Total de regs: ', len(bd))
vacios = [[]]
for colum in cols:
    vacios[0].append(len(bd[pd.isna(bd[colum])]))

print(vacios)

##########################Edad_c##########################
#0: No especificado
#9: Se ignora
tipoEdadNoEp = len(bd[(bd.edad_c == 0) | (bd.edad_c == 9)])
print('Edad no especificada: ', tipoEdadNoEp)
vacios[0][0] += tipoEdadNoEp
##########################Edad##########################
#Edades de 0 a 130 años
#999: Desconocido 
edadNoEp = len(bd[(bd.edad == 999)])
print('Edad desconocida: ', edadNoEp)
vacios[0][1] += edadNoEp

##########################Nom_c##########################
#[Desconocido Desconocido Desconocido], [x x x], [Desconocida Desconocida Desconocida], [x x Desconocido Trece], [cuatro jur seis Desconocido]
#[S/A], [SN SN SN]
#vacios[0][2] += tipoEdadNoEp

##########################Nac_Dia##########################
#99: Desconocido
diaNac_NoEp = len(bd[(bd.nac_dia == 99)])
print('Dia_Nacimiento desconocido: ', diaNac_NoEp)
vacios[0][3] += diaNac_NoEp

##########################Nac_Mes##########################
#99: Desconocido
mesNac_NoEp = len(bd[(bd.nac_mes == 99)])
print('Mes_Nacimiento desconocido: ', mesNac_NoEp)
vacios[0][4] += mesNac_NoEp

##########################Nac_Anio##########################
#99: Desconocido
anioNac_NoEp = len(bd[(bd.nac_anio == 9999)])
print('Anio_Nacimiento desconocido: ', anioNac_NoEp)
vacios[0][5] += anioNac_NoEp

##########################Sexo##########################
#0: No especificado
#9: Se ignora
sexo_NoEp = len(bd[(bd.sexo == 0) | (bd.sexo == 9)])
print('Sexo desconocido: ', sexo_NoEp)
vacios[0][6] += sexo_NoEp

##########################Afiliacion##########################
#0,1,2,3,4,99
#vacios[0][7] += sexo_NoEp

##########################curp##########################
sinCURP = bd[(pd.isna(bd['curp']) == False)]
conCURP = sinCURP[(sinCURP['curp'].str.contains('^[A-Z]{4}[0-9]{2}(0[1-9]|1[0-2])(0[1-9]|1[0-9]|2[0-9]|3[0-1])[A-Z]{6}[0-9A-Z]{1}[0-9]{1}', case=False))]
total = len(sinCURP) - len(conCURP)
vacios[0][8] += total

##########################edo_civil (id_conyugal)##########################
#0: No especificado
#9: Se ignora
edoCiv_NoEp = len(bd[(bd.edo_civil == 0) | (bd.edo_civil == 9)])
print('Estado civ desconocido: ', edoCiv_NoEp)
vacios[0][9] += edoCiv_NoEp

##########################Escolar##########################
#0: No especificado
#99: Se ignora
escolar_NoEp = len(bd[(bd.escolar == 0) | (bd.escolar == 99)])
print('Escolaridad desconocida: ', escolar_NoEp)
vacios[0][10] += escolar_NoEp

##########################ocupacion##########################
#0: No especificado
#99: ocupaciones no especificadas
#999: Se ignora
Ocup_NoEp = len(bd[(bd.ocupacion == 0) | (bd.ocupacion == 99) | (bd.ocupacion == 999)])
print('Ocupaciones desconocidas: ', Ocup_NoEp)
vacios[0][11] += Ocup_NoEp



Total de regs:  619561
[[0, 0, 150, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
Edad no especificada:  1225
Edad desconocida:  1195
Dia_Nacimiento desconocido:  4119
Mes_Nacimiento desconocido:  4110
Anio_Nacimiento desconocido:  3957
Sexo desconocido:  1520


C:\Users\Hapy_\AppData\Local\Temp\ipykernel_4168\2073334954.py:58: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  conCURP = sinCURP[(sinCURP['curp'].str.contains('^[A-Z]{4}[0-9]{2}(0[1-9]|1[0-2])(0[1-9]|1[0-9]|2[0-9]|3[0-1])[A-Z]{6}[0-9A-Z]{1}[0-9]{1}', case=False))]


Estado civ desconocido:  25460
Escolaridad desconocida:  22426
Ocupaciones desconocidas:  0


In [ ]:
#bd[(pd.isna(bd['nom_c']) == False) & (bd['nom_c'] != 'DESCONOCIDO DESCONOCIDO DESCONOCIDO') & (bd['nom_c'] != 'DESCONOCIDA DESCONOCIDA DESCONOCIDA')]
#(pd.isna(bd['nom_c']) == False)


sinNaN = bd[(pd.isna(bd['nom_c']) == False)]
sinDesconocido = sinNaN[(sinNaN['nom_c'].str.contains('DESCONOCIDO', case=False) == False) & (sinNaN['nom_c'] != 'DESCONOCIDO DESCONOCIDO DESCONOCIDO')]
sinDesconocida = sinDesconocido[(sinDesconocido['nom_c'].str.contains('DESCONOCIDA', case=False) == False) & (sinDesconocido['nom_c'] != 'DESCONOCIDA DESCONOCIDA DESCONOCIDA')]
#n
sinN = sinDesconocida[(sinDesconocida['nom_c'].str.contains('^N N (N$|FEMENINO$|MASCULINO$|RECIEN NACIDO$|RN$|R N$)', case=False)) == False]
#x
sinX = sinN[(sinN['nom_c'].str.contains('^x X (RECIEN NACIDO$|RN$|X$|R N$|FEMENINO$|MASCULINO$|N$)', case=False)) == False]
#sn, sr
sinSN = sinX[(sinX['nom_c'].str.contains('^sn sn (RECIEN NACIDO$|RN$|X$|R N$|FEMENINO$|MASCULINO$|N$|sn$)', case=False)) == False]
#sinX[(sinX['nom_c'].str.contains('^sr sr (RECIEN NACIDO$|RN$|X$|R N$|FEMENINO$|MASCULINO$|N$|sr$)', case=False))]
#SIN INFORMACION
#sinSN[(sinSN['nom_c']=='MARTINEZ BECERRA SIN INFORMACION')]
#rn, r n, niña, niño (recien nacido)
#sinSN[(sinSN['nom_c'].str.contains('niño', case=False))]

#sinSN.to_csv('nomLimpios.csv', header=True, index=False)

########################################################################
#sinDesconocida[(sinDesconocida['nom_c'].str.contains('s/a', case=False))]
#sinDesconocida[(sinDesconocida['nom_c'].str.contains('^N N (N$|FEMENINO$|MASCULINO$|RECIEN NACIDO$|RN$|R N$)', case=False))]
#sinX[(sinX['nom_c'].str.contains('sr ', case=False))]
#sinSN[(sinSN['nom_c'].str.contains('SIN INFORMACION', case=False))]

C:\Users\Hapy_\AppData\Local\Temp\ipykernel_4168\3551163828.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sinN = sinDesconocida[(sinDesconocida['nom_c'].str.contains('^N N (N$|FEMENINO$|MASCULINO$|RECIEN NACIDO$|RN$|R N$)', case=False)) == False]
C:\Users\Hapy_\AppData\Local\Temp\ipykernel_4168\3551163828.py:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sinX = sinN[(sinN['nom_c'].str.contains('^x X (RECIEN NACIDO$|RN$|X$|R N$|FEMENINO$|MASCULINO$|N$)', case=False)) == False]
C:\Users\Hapy_\AppData\Local\Temp\ipykernel_4168\3551163828.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sinSN = sinX[(sinX['nom_c'].str.contains('^sn sn (RECIEN NACIDO$|RN$|X$|R N$|FEMENINO$|MASCULINO$|N$|sn$)', case=False)) == False]

In [ ]:
len(sinSN)

vacios[0][2] = (len(bd) - len(sinSN))

datavacios = pd.DataFrame(vacios, columns=cols)
datavacios

618617

,edad_c,edad,nom_c,nac_dia,nac_mes,nac_anio,sexo,afiliacion,curp,edo_civil,escolar,ocupacion
0,1225,1195,944,4119,4110,3957,1520,0,402536,25460,22426,0


In [ ]:

#residencia = ['res_ent', 'res_mun', 'res_loc', 'def_ent', 'def_mun', 'def_sitio', 'def_dia', 'def_mes', 'def_anio']
##########################Campos vacios##########################
print('Total de regs: ', len(bd))
vaciosRes = [[]]
for colum in residencia:
    vaciosRes[0].append(len(bd[pd.isna(bd[colum])]))

print('Res vacios = ', vaciosRes)

##########################res_ent##########################
#00: No especificado   33: EUA
#88: No aplica         34: Otros paises latinos  36: Mexico
#99: Se ignora         35: Otros paises
resEntNoEp = len(bd[(bd.res_ent == 0) | (bd.res_ent == 88) | (bd.res_ent == 99)])
print('Entidad de residencia desconocida: ', resEntNoEp)
vaciosRes[0][0] += resEntNoEp
##########################res_mun##########################
#01998, 02998, ..., 32998: Se Ignora
#01999, 02999, ..., 32999: No Espec
#88997, 97997: No Aplica
#99998, 99999, 00999: No Espec 
#98998: Se Ignora
munNoEp = len(bd[((bd.res_mun%1000) == 999) | ((bd.res_mun%1000) == 998) | ((bd.res_mun%1000) == 997)])
print('Municipios de residencia desconocidos: ', munNoEp)
vaciosRes[0][1] += munNoEp
##########################res_loc##########################
#999999999: ?
locNoEp = len(bd[(bd.res_loc == 999999999)])
print('localidades de residencia desconocidos: ', locNoEp)
vaciosRes[0][2] += locNoEp
##########################def_ent##########################
#00: No especificado   33: EUA
#88: No aplica         34: Otros paises latinos  36: Mexico
#99: Se ignora         35: Otros paises
defEntNoEp = len(bd[(bd.def_ent == 0) | (bd.def_ent == 88) | (bd.def_ent == 99)])
print('Entidad de defuncion desconocida: ', defEntNoEp)
vaciosRes[0][3] += defEntNoEp
##########################def_mun##########################
#01998, 02998, ..., 32998: Se Ignora
#01999, 02999, ..., 32999: No Espec
#88997, 97997: No Aplica
#99998, 99999, 00999: No Espec 
#98998: Se Ignora
defmunNoEp = len(bd[((bd.def_mun%1000) == 999) | ((bd.def_mun%1000) == 998) | ((bd.def_mun%1000) == 997)])
print('Municipios de residencia desconocidos: ', defmunNoEp)
vaciosRes[0][4] += defmunNoEp
##########################def_sitio##########################
#99: Se ignora
#00: no especificado
#sitioNoEp = len(bd[(bd.def_sitio == 99)] | (bd.def_sitio == 0))
#print('localidades de residencia desconocidos: ', sitioNoEp)
#vaciosRes[0][5] += sitioNoEp
##########################def_dia##########################
#99: Desconocido
diaDef_NoEp = len(bd[(bd.def_dia == 99)])
print('Dia_Defuncion desconocido: ', diaDef_NoEp)
vaciosRes[0][6] += diaDef_NoEp

##########################def_mes##########################
#99: Desconocido
mesDef_NoEp = len(bd[(bd.def_mes == 99)])
print('Mes_Defuncion desconocido: ', mesDef_NoEp)
vaciosRes[0][7] += mesDef_NoEp

##########################def_anio##########################
#99: Desconocido
anioDef_NoEp = len(bd[(bd.def_anio == 9999)])
print('Anio_Defuncion desconocido: ', anioDef_NoEp)
vaciosRes[0][8] += anioDef_NoEp


print('Res vacios = ', vaciosRes)

datavaciosRes = pd.DataFrame(vaciosRes, columns=residencia)
datavaciosRes

Total de regs:  619561
Res vacios =  [[0, 0, 0, 0, 0, 0, 0, 0, 0]]
Entidad de residencia desconocida:  3916
Municipios de residencia desconocidos:  2227
localidades de residencia desconocidos:  1971
Entidad de defuncion desconocida:  209
Municipios de residencia desconocidos:  137
Dia_Defuncion desconocido:  375
Mes_Defuncion desconocido:  338
Anio_Defuncion desconocido:  325
Res vacios =  [[3916, 2227, 1971, 209, 137, 0, 375, 338, 325]]


,res_ent,res_mun,res_loc,def_ent,def_mun,def_sitio,def_dia,def_mes,def_anio
0,3916,2227,1971,209,137,0,375,338,325


In [ ]:
#vacios + vaciosRes
vacios[0].insert(0, len(bd))
#vacios[0].pop(0)
nombreSav = 'def' + anio + '.csv'
def20xx = pd.DataFrame([vacios[0] + vaciosRes[0]], columns=['Total'] + cols + residencia)

In [ ]:
def20xx.to_csv('./dgis_datos/defuncion/faltantes/' + nombreSav, header=True, index=False)